In [1]:
import ee
import folium
from folium import plugins
from flood_mapper import derive_flood_extents, export_flood_data

In [ ]:
import flood_mapper

In [2]:
ee.Authenticate()
ee.Initialize()

## Input parameters

In [3]:
# Define a start and end date between to select imagery before the flooding event
before_start = '2020-10-01'
before_end = '2020-10-15'

# Define a start and end date between to select imagery after the flooding event
after_start = '2020-11-04'
after_end = '2020-11-15'

# Define a geographic region where the flooding occurred.
region = ee.Geometry.Polygon([[[-85.93, 16.08],
                               [-85.93, 15.69],
                               [-85.40, 15.69],
                               [-85.40, 16.08]]])




## Perform flood detection

In [4]:
# Change the export flag to 'False' if you do not wish to export the results to Google Drive
detected_flood_vector, detected_flood_raster, imagery = derive_flood_extents(region,
                                                                             before_start,
                                                                             before_end,
                                                                             after_start,
                                                                             after_end,
                                                                             export=False)

## View the results

#### Firstly, set up folium to be able to view GEE layers

In [5]:
# Add basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#### Display the results

In [6]:
location = region.centroid().coordinates().getInfo()[::-1]

# Set visualization parameters.
s1_vis_params = {'min': -30, 'max': 0}
flood_vis_params = {'min': 0, 'max': 1, 'palette':'blue'}

# Create a folium map object and add basemaps
flood_map = folium.Map(location=location, zoom_start=12, height=500)
basemaps['Google Maps'].add_to(flood_map)
basemaps['Google Satellite Hybrid'].add_to(flood_map)

# Add the results to the map.
flood_map.add_ee_layer(imagery, s1_vis_params, 'S1 image after flood')
flood_map.add_ee_layer(detected_flood_raster, flood_vis_params, 'Detected flood extents')

# Add a layer control panel to the map.
flood_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(flood_map)

# Display the map.
display(flood_map)

## Export the results to Google Drive

In [8]:
# export_flood_data(detected_flood_vector, detected_flood_raster, imagery, region, 'your_filename')